In [1]:
from __future__ import print_function
import os
import tensorflow as tf
import numpy as np
import sklearn
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report

from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Activation, \
                         Dropout, MaxPooling1D, GlobalAveragePooling1D, \
                         GlobalMaxPooling1D, Lambda, Concatenate, Dense, regularizers
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers, activations
from keras.optimizers import Adam

from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report


os.environ["CUDA_VISIBLE_DEVICES"]="1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)
print("Using GPU: ", K.tensorflow_backend._get_available_gpus())

K.set_image_dim_ordering('tf')

Using TensorFlow backend.


Using GPU:  ['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
def retrieve_file(file_name):
    path = './'
    outfile = path + file_name
    X = np.load(outfile)
    X = X['arr_0']
    return X

def retrieve_file_no_aug(file_name):
    path = '../data/raw_data/processed_topic/'
    outfile = path + file_name
    X = np.load(outfile, encoding='latin1')
    X = X['arr_0']
    return X

In [3]:
def standard_confusion_matrix(y_test, y_test_pred):
    """
    Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    Parameters
    ----------
    y_true : ndarray - 1D
    y_pred : ndarray - 1D

    Returns
    -------
    ndarray - 2D
    """
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    print("\nConfusion Matrix: (sklearn)\n")
    print(cnf_matrix)
    [[tn, fp], [fn, tp]] = cnf_matrix
    return np.array([[tp, fp], [fn, tn]])


def model_performance(model, X_train, X_test, y_train, y_test):
    """
    Evaluation metrics for network performance.
    """
    y_test_pred = np.argmax(model.predict(X_test), axis=-1)
    y_train_pred = np.argmax(model.predict(X_train), axis=-1)

    # Computing confusion matrix for test dataset
    conf_matrix = standard_confusion_matrix(y_test, y_test_pred)
    print("\nConfusion Matrix:\n")
    print(conf_matrix)
    
    target_name=['non-depressed','depressed']
    clf_report = classification_report(y_test, y_test_pred, target_names=target_name)
    print("\nClassification Report (sklearn):\n")
    print(clf_report)
    
    print("\nF1-Score: {}".format(sklearn.metrics.f1_score(y_test, y_test_pred)))
    return y_train_pred, y_test_pred, conf_matrix, clf_report

In [4]:
train_audio_features = np.loadtxt("train_audio_features_augm200.txt")
print(train_audio_features.shape)
test_audio_features = np.loadtxt("test_audio_features_augm200.txt")
print(test_audio_features.shape)

train_text_features = np.loadtxt("train_text_features_augm200.txt")
print(train_text_features.shape)
test_text_features = np.loadtxt("test_text_features_augm200.txt")
print(test_text_features.shape)

train_features_audio_text = np.concatenate((train_text_features,train_audio_features),axis=1)
print(train_features_audio_text.shape)
test_features_audio_text = np.concatenate((test_text_features,test_audio_features),axis=1)
print(test_features_audio_text.shape)

y_train_no_aug = retrieve_file_no_aug('train_labels.npz')
y_train3 = retrieve_file('train_labels_200.npz')

y_train=np.concatenate((y_train_no_aug, y_train3), axis=0)
y_test = retrieve_file_no_aug('test_labels.npz')

nb_classes=2


# Convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


(307, 64)
(33, 64)
(307, 512)
(33, 512)
(307, 576)
(33, 576)


In [5]:
from keras.models import load_model

model = load_model('fd_augm_run200_f10.869565217391.h5')
model.compile(loss='categorical_crossentropy',
                 optimizer=Adam(lr=6.25e-5),
                 metrics=['accuracy'])
model.summary()

#score_train = model.evaluate(X_train, y_train, verbose=0)
# print('Train accuracy:', score_train[1])
# score_test = model.evaluate(X_test, y_test, verbose=0)
# print('Test accuracy:', score_test[1])
# evaluate model
print('\nEvaluating model...')
y_train_pred, y_test_pred, conf_matrix, clf_report = model_performance(model, train_features_audio_text, test_features_audio_text, y_train, y_test)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2268 (InputLayer)      (None, 576)               0         
_________________________________________________________________
dense_5181 (Dense)           (None, 64)                36928     
_________________________________________________________________
batch_normalization_2914 (Ba (None, 64)                256       
_________________________________________________________________
activation_2914 (Activation) (None, 64)                0         
_________________________________________________________________
dropout_2914 (Dropout)       (None, 64)                0         
_________________________________________________________________
dense_5182 (Dense)           (None, 64)                4160      
_________________________________________________________________
batch_normalization_2915 (Ba (None, 64)                256       
__________